#### just opening the netcdf `dump=True` from `Cyclone_damage_funcs.py`
#### then saving out as geotiff 

In [1]:
import glob
import numpy as np
import xarray as xr
import rioxarray
import datacube
from datacube.testutils.io import rio_slurp_xarray
from datacube.utils.cog import write_cog

In [2]:
cyclone_name = 'Yasi'

#### immediate impacts (cyclone year before and cyclone year after)

#### long term impacts

In [3]:
# all = long term impacts
nc_all = (glob.glob('./'+cyclone_name+'/*_all.nc'))
print(glob.glob('./'+cyclone_name+'/*_all.nc'))

['./Yasi/Yasi_138_-191_all.nc', './Yasi/Yasi_144_-201_all.nc', './Yasi/Yasi_144_-211_all.nc', './Yasi/Yasi_144_-191_all.nc', './Yasi/Yasi_153_-219_all.nc', './Yasi/Yasi_144_-219_all.nc']


In [ ]:
# nc_ds = xr.open_dataset('./'+cyclone_name+'/Yasi_144_-211_all.nc')
# singleband_da = nc_ds.damage_level.isel(time=0)
# # singleband_da.attrs['crs'] = 'EPSG:3577'

In [ ]:
# nc_ds

In [5]:
# singleband_da.attrs

{'grid_mapping': 'spatial_ref'}

In [ ]:
# write_cog(geo_im=singleband_da, fname='Yasi_144_-211_all.tif',
#               overwrite=True)

In [ ]:
# singleband_da.rio.to_raster(raster_path="Yasi_144_-211_all_rio.tif", driver='GTiff')

In [ ]:
# nc_ds = xr.open_dataset('Yasi_144_-211_all.tif')
# nc_ds

In [4]:
for i in nc_all:
    # open dataset
    nc_ds = xr.open_dataset(i)
    # select time=0
    singleband_da = nc_ds.damage_level.isel(time=0)
    # print name and unique values in dataset
    print(i[7:-3])
    print(np.unique(singleband_da.values))
    # Write GeoTIFF to a location
    write_cog(geo_im=singleband_da,
              fname=i[7:-3]+'.tif',
              overwrite=True)


Yasi_138_-191_all
[-1]
Yasi_144_-201_all
[-1  0  1  2  3  4  5]
Yasi_144_-211_all
[-1  0  1  2  3  4  5]
Yasi_144_-191_all
[-1  0  1  3  4  5]
Yasi_153_-219_all
[-1  0  1  2  3  4  5]
Yasi_144_-219_all
[-1  0  1  2  3  4]


In [ ]:
'''
commandline gdal to get mosaic

# _all = long term damage


# adjust NS resolution 
for file in *.tif; 
    do echo "$file"; gdalwarp -t_srs EPSG:3577 -dstnodata -9999 -overwrite "$file" "$(basename "$file" .tif)_.tif"
done

# create vrt
gdalbuildvrt -o Yasi_all.vrt *_.tif

# make into tif
gdal_translate -of GTiff -co "COMPRESS=LZW" -co "TILED=YES" -co "BIGTIFF=YES" Yasi_all.vrt Yasi_all.tif

# open .tif in GIS
'''